In [13]:
import mediapipe as mp
import cv2

In [14]:
# inisialiasai mediapipe
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

In [15]:
cap = cv2.VideoCapture(0) #menampilkan webcam dengan kamera pertama atau ke-0 (kamera default)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 720)

True

In [16]:
def jari_terbuka(landmarks):
    ibu_jari = 1 if landmarks[4][0] > landmarks[3][0] else 0
    telunjuk = 1 if landmarks[8][1] < landmarks[6][1] else 0
    tengah   = 1 if landmarks[12][1] < landmarks[10][1] else 0
    manis    = 1 if landmarks[16][1] < landmarks[14][1] else 0
    kelingking = 1 if landmarks[20][1] < landmarks[18][1] else 0
    return [ibu_jari, telunjuk, tengah, manis, kelingking]

In [ ]:
# inisialisasi detektor dari mediapipe
with mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as hands :

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("gagal membaca frame dari webcam")
            break

        frame = cv2.flip(frame, 1)

        # ubah warna dari BGR (default opencv) ke RGB (mediapipe)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False # meningkatkan performa

        # proses deteksi tangan
        result = hands.process(image)

        #kembali ke BGR untuk ditampilkan dengan opencv
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


        #gambar landmark jika terdeteksi
        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS
                )

                # Ambil koordinat setiap titik
                landmarks = []
                for lm in hand_landmarks.landmark:
                    landmarks.append((lm.x, lm.y))

                # Cek status jari
                status_jari = jari_terbuka(landmarks)

                # Tampilkan status di layar
                if all(status_jari):
                    cv2.putText(image, "Tangan Terbuka", (10, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                elif not any(status_jari):
                    cv2.putText(image, "Tangan Mengepal", (10, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                else:
                    cv2.putText(image, "    Setengah Terbuka", (10, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        #menampilkan webcam ke layar
        cv2.imshow("testes", image)

        #tekan q untuk keluar
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


#tuutup kameran dan jendela
cap.release()
cv2.destroyAllWindows()